In [1]:
import insightface
import warnings
warnings.simplefilter('ignore')

C:\Users\nitin\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from insightface.app import FaceAnalysis
app_l=FaceAnalysis(name='buffalo_l',root='insightface_model')

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5


In [3]:
app_l.prepare(ctx_id=0,det_size=(640,640))

set det-size: (640, 640)


In [4]:
import cv2
img=cv2.imread('insightface/test_image_1.jpg')
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
results_l=app_l.get(img)

C:\Users\nitin\anaconda3\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [6]:
print(results_l)

[{'bbox': array([908.9962  , 120.859375, 983.00574 , 215.04736 ], dtype=float32), 'kps': array([[931.1319 , 153.45071],
       [964.7412 , 153.1146 ],
       [948.6931 , 171.61122],
       [931.7916 , 186.40295],
       [963.45844, 186.2429 ]], dtype=float32), 'det_score': 0.93273956, 'landmark_3d_68': array([[ 9.0746741e+02,  1.5839578e+02,  5.0243629e+01],
       [ 9.0944232e+02,  1.6857220e+02,  4.7775455e+01],
       [ 9.1119391e+02,  1.7897560e+02,  4.4467281e+01],
       [ 9.1362549e+02,  1.8885023e+02,  3.9394665e+01],
       [ 9.1758362e+02,  1.9743614e+02,  3.0614346e+01],
       [ 9.2296228e+02,  2.0369115e+02,  2.3594717e+01],
       [ 9.2881738e+02,  2.0797414e+02,  1.7070038e+01],
       [ 9.3632129e+02,  2.1072058e+02,  8.4526978e+00],
       [ 9.4872974e+02,  2.1347729e+02,  3.9831481e+00],
       [ 9.6010333e+02,  2.1096448e+02,  1.1198088e+01],
       [ 9.6714709e+02,  2.0717723e+02,  2.2069340e+01],
       [ 9.7237146e+02,  2.0174071e+02,  3.1079845e+01],
       [ 9.7

In [7]:
len(results_l)

113

In [8]:
results_l[0].keys()

dict_keys(['bbox', 'kps', 'det_score', 'landmark_3d_68', 'pose', 'landmark_2d_106', 'gender', 'age', 'embedding'])

In [9]:
results_l[0].kps

array([[931.1319 , 153.45071],
       [964.7412 , 153.1146 ],
       [948.6931 , 171.61122],
       [931.7916 , 186.40295],
       [963.45844, 186.2429 ]], dtype=float32)

In [10]:
results_l[0].det_score

0.93273956

In [11]:
img_copy=img.copy()
for r in results_l:
    x1,y1,x2,y2=r['bbox'].astype('int')
    
    cv2.rectangle(img_copy,(x1,y1),(x2,y2),(0,255,0),1)
    
    kps=r['kps'].astype('int')
    for k1,k2 in kps:
        cv2.circle(img_copy,(k1,k2),2,(255,255,0),-1)
    
    score="score: {}%".format(int(r['det_score']*100))
    cv2.putText(img_copy,score,(x1,y1),cv2.FONT_HERSHEY_PLAIN,1,(255,255,255))
    
    gender_en=['Female','Male']
    gender=gender_en[r['gender']]
    age=r['age']
    age_gender=f"{gender}::{age}"
    cv2.putText(img_copy,age_gender,(x2,y2),cv2.FONT_HERSHEY_PLAIN,1,(255,255,255))
    
cv2.imshow('bbox',img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
import re
import os
import cv2
person_info=[]
def clean_name(string):
    string=re.sub(r'[^A-Za-z]',' ',string)
    string=string.title()
    return string

path='D:\PDPU Assignments\Internship\8Sem Internship\project\data\images'
listdir=os.listdir(path=path)
for folder in listdir:
    role, name= folder.split('-')
    name=clean_name(name)
    role=clean_name(role)
    #print(role,name)
    
    img_files=os.listdir(path=f'{path}/{folder}')
    #print(img_files)
    
    for file in img_files:
        p=f'{path}/{folder}/{file}'
        img=cv2.imread(p)
        result=app_l.get(img)
        if len(result)>0:
            res=result[0]
            embedding=res['embedding']
            person_info.append([name,role,embedding])

In [5]:
import pandas as pd
df=pd.DataFrame(person_info,columns=['Name','Role','Facial_Features'])
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.07824361, 1.9015113, -0.023395985, -0.93351..."
1,Angelina Jolie,Student,"[0.2310524, 0.97989607, -0.72453475, -0.320673..."
2,Angelina Jolie,Student,"[0.07103528, 2.1639588, -0.3409844, -0.7473231..."
3,Angelina Jolie,Student,"[0.4855191, 2.114588, 0.4463554, -0.664585, -0..."
4,Angelina Jolie,Student,"[-0.08823502, 1.5141321, -0.18750465, -1.34230..."
5,Angelina Jolie,Student,"[-0.54577935, 1.5319035, -0.12923717, -1.15202..."
6,Angelina Jolie,Student,"[0.9446974, 1.6263472, -0.25197536, -1.4123584..."
7,Angelina Jolie,Student,"[0.42334622, 0.33403736, 0.12036865, 0.0760047..."
8,Angelina Jolie,Student,"[-0.36375806, 1.3726236, 0.06761645, -0.857597..."
9,Angelina Jolie,Student,"[0.7520716, 1.2098383, 0.043703765, -1.4506797..."


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
def search(df,feature_col,test_vector,threshold=0.5):
    df=df.copy()
    x_list=df[feature_col].to_list()
    x=np.asarray(x_list)
    
    similar=cosine_similarity(x,test_vector.reshape(1,-1))
    similar_arr=np.array(similar).flatten()
    df['cosine']=similar_arr
    
    data_filter=df.query(f'cosine >= {threshold}')
    if len(data_filter)>0:
        data_filter.reset_index(drop=True,inplace=True)
        argmax=data_filter['cosine'].argmax()
        person_name,person_role=data_filter.loc[argmax][['Name','Role']]
        
    else:
        person_name='Unknown'
        person_role='Unknown'

    return person_name,person_role

In [7]:
import cv2
test_image=cv2.imread('insightface/test_11.jpg')
cv2.imshow('test image',test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
results=app_l.get(test_image)
test_copy=test_image.copy()
import numpy as np
for r in results:
    x1,y1,x2,y2=r['bbox'].astype('int')
    embeddings=r['embedding']
    person_name,person_role=search(df,'Facial_Features',embeddings,threshold=0.5)
    
    if person_name=='Unknown':
        color=(0,0,255)
    else:
        color=(0,255,0)
    cv2.rectangle(test_copy,(x1,y1),(x2,y2),color) 
    cv2.putText(test_copy,person_name,(x1,y1),cv2.FONT_HERSHEY_DUPLEX,0.7,color,2)
    
cv2.imshow('test image',test_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
df_mean=df.groupby(by=['Name','Role']).mean()

In [11]:
df_mean.reset_index(inplace=True)

In [12]:
results=app_l.get(test_image)
test_copy=test_image.copy()
import numpy as np
for r in results:
    x1,y1,x2,y2=r['bbox'].astype('int')
    embeddings=r['embedding']
    person_name,person_role=search(df_mean,'Facial_Features',embeddings,threshold=0.5)
    
    if person_name=='Unknown':
        color=(0,0,255)
    else:
        color=(0,255,0)
    cv2.rectangle(test_copy,(x1,y1),(x2,y2),color) 
    cv2.putText(test_copy,person_name,(x1,y1),cv2.FONT_HERSHEY_DUPLEX,0.7,color,2)
    
cv2.imshow('test image',test_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
xvalues=df_mean.values
cols=np.array(df_mean.columns)
cols

array(['Name', 'Role', 'Facial_Features'], dtype=object)

In [14]:
np.savez('df_students_teacher.npz',xvalues,cols)

In [15]:
file_np=np.load('df_students_teacher.npz',allow_pickle=True)
file_np.files

['arr_0', 'arr_1']

In [17]:
pd.DataFrame(file_np['arr_0'],columns=file_np['arr_1'])

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.19881932, 1.4748837, -0.0979588, -0.8805054..."
1,Barack Obama,Teacher,"[-0.22177827, 1.5762169, -0.5030781, 0.1360128..."
2,Chris Evans,Student,"[0.35691047, -0.41491055, 0.039218444, -0.1543..."
3,Morgan Freeman,Teacher,"[0.76278937, 0.890615, 0.46873552, 0.25898102,..."
4,Scarlett Johansson,Student,"[-0.13535985, -0.2821385, -0.42700845, -0.2825..."
